In [34]:
from collections import defaultdict
   
def create_features(X,Y):
    phi=defaultdict(int)
    for i in range(len(Y)+1):
        if i==0:
            first_tag="<s>"
        else:
            first_tag=Y[i-1]
        if i==len(Y):
            next_tag="</s>"
        else:
            next_tag=Y[i]
        phi[f"T,{first_tag},{next_tag}"]+=1
    for i in range(len(Y)):
        phi[f"E,{Y[i]},{X[i]}"]+=1
        phi[f"CAPS,{Y[i]}"]+=1
    return phi

def HMM_viterbi(w,words,possible_tags,transition):
    l=len(words)
    best_score={"0 <s>":0}
    best_edge={"0 <s>":None}
    for i in range(l):
        for prev in possible_tags:
            for next_ in possible_tags:
                if f"{i} {prev}" in best_score and f"{prev} {next_}" in transition:
                    score=best_score[f"{i} {prev}"]+w[f"T,{prev},{next_}"]+w[f"E,{next_},{words[i]}"]
                    if f"{i+1} {next_}" not in best_score or best_score[f"{i+1} {next_}"]<score:
                        best_score[f"{i+1} {next_}"]=score
                        best_edge[f"{i+1} {next_}"]=f"{i} {prev}"
    for tag in possible_tags:
        if f"{tag} <\s>" in transition:
            score=best_score[f"{l} {tag}"]+w[f"T,{tag},<\s>"]
            if f"{l+1} <\s>" not in best_score or best_score[f"{l+1} <\s>"]<score:
                best_score[f"{l+1} <\s>"]=score
                best_edge[f"{l+1} <\s>"]=f"{l} {tag}"
            
    tag_path=[]
    next_edge=best_edge[f"{l+1} <\s>"]
    while next_edge!="0 <s>":
        no,tag=next_edge.split()
        tag_path.append(tag)
        next_edge=best_edge[next_edge]
    tag_path.reverse()
    
    return tag_path
            
            
#train
with open("wiki-en-train.norm_pos.txt","r",encoding="utf-8") as f:
    data=[]
    for line in f:
        word_pos=line.strip().split()
        X=[]
        Y=[]
        for w_p in word_pos:
            x,y=w_p.split("_")
            X.append(x)
            Y.append(y)
        data.append((X,Y))
        
    transition=defaultdict(int)
    possible_tags={"<s>","<\s>"}
    for X,Y in data:
        Y_s=["<s>"]+Y+["<\s>"]
        for i in range(len(Y_s)-1):
            transition[f"{Y_s[i]} {Y_s[i+1]}"]+=1
        possible_tags.update(Y_s)
    
    w=defaultdict(int)
    iterations=1
    for num in range(iterations):
        for X,Y_prime in data:
            Y_hat=HMM_viterbi(w,X,possible_tags,transition)
            phi_prime=create_features(X,Y_prime)
            phi_hat=create_features(X,Y_hat)
            for key in phi_prime:
                w[key]+=phi_prime[key]
            for key in phi_hat:
                w[key]-=phi_hat[key]
                
#test
with open("wiki-en-test.norm.txt","r",encoding="utf-8") as f1:
    for line in f1:
        X_test=line.strip().split()
        Y_hat_test=HMM_viterbi(w,X_test,possible_tags,transition)
        with open("answer.txt","a",encoding="utf-8") as ans:
            ans.write(" ".join(Y_hat_test)+"\n")

In [35]:
with open("answer.txt","r",encoding="utf-8") as ans:
    Y_test=[]
    for line in ans:
        tags=line.strip().split()
        Y_test.append(tags)

with open("wiki-en-test.norm_pos.txt","r",encoding="utf-8") as f2:
    Y_ans=[]
    for line in f2:
        word_pos=line.strip().split()
        Y=[]
        for w_p in word_pos:
            x,y=w_p.split("_")
            Y.append(y)
        Y_ans.append(Y)
    
    count=0
    correct=0
    for test,ans in zip(Y_test,Y_ans):
        for i,j in zip(test,ans):
            count+=1
            if(i==j):
                correct+=1
    print(str(correct/count))

0.811965811965812
